In [1]:
import pandas as pd
import os 
import glob

In [2]:
# Change this year to match the excel file you are adding
year = '2015'

In [3]:
# Imports the running dataset which the year selected will be added
if os.path.isfile('cumltowntaxincome.csv'):
    cumltowntax = pd.read_csv("cumltowntaxincome.csv")
else:
    cumltowntax =  pd.DataFrame()
    
if os.path.isfile('cumltowntotal.csv'):
    cumltowntotal = pd.read_csv("cumltowntotal.csv")
else:
    cumltowntotal = pd.DataFrame()


In [4]:
if cumltowntax.empty:
    print('New Town Dataframe Created')
else:
    print("Years already stored for Towns:" + str(pd.unique(cumltowntax['Year'])))
    
if cumltowntotal.empty:
    print('New County Dataframe Created')
else:
    print("Years already stored for County:" + str(pd.unique(cumltowntotal['Year'])))

Years already stored for Towns:[2005 2006 2007 2008 2009 2010 2011 2012 2013 2014]
Years already stored for County:[2005 2006 2007 2008 2009 2010 2011 2012 2013 2014]


In [5]:
glob.glob('./*.xls')

['.\\townGEOIDs.xls',
 '.\\town_income_2005_detail_report.xls',
 '.\\town_income_2006_detail_report.xls',
 '.\\town_income_2007_detail_report.xls',
 '.\\town_income_2008_detail_report.xls',
 '.\\town_income_2009_detail_report.xls',
 '.\\town_income_2010_detail_report.xls',
 '.\\town_income_2011_detail_report.xls',
 '.\\town_income_2012_detail_report.xls',
 '.\\town_income_2013_detail_report.xls',
 '.\\town_income_2014_detail_report.xls',
 '.\\town_income_2015_detail_report.xls']

In [6]:
# Imports excel file and gives list of sheets
data = pd.ExcelFile('town_income_'+year+'_detail_report.xls')
colList = data.sheet_names
print(colList)

['Sheet1']


In [7]:
# Parse each sheet into it's own dataframes
towntax = data.parse('Sheet1',  skiprows=2)

## Rename Columns

Because shapefiles, the final format for this data, has a limit of 10 characters for headers, it makes sense to abbreviate this now both to save on typing out the names and to make it friendly when joining it later. 

In [8]:
towntaxCol = ['AGIClss',  'Return', 'Exempt', 'MarJnt', 'Single','MarSep', 'HdHous', 'AGI',
               'FedTI', 'VTI', 'NetVTax']

towntax.columns = towntaxCol

In [9]:
# There are some footnotes in the excel file. Important to note but they are cut from the final file at the end 
towntax.tail()

,AGIClss,Return,Exempt,MarJnt,Single,MarSep,HdHous,AGI,FedTI,VTI,NetVTax
4286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4289,*Indicates 10 or fewer returns and data has be...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4290,Towns with no reportable data were excluded fr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Remove the several blank rows from the spreadsheet
def removeNA(dfname):
    dfname = dfname[(dfname.iloc[:,0].notnull())]
    return dfname
 
towntax = removeNA(towntax)
towntax = towntax.reset_index(drop=True)

## Add Columns and Town Names

The original excel files did not place the county name on each line but rather at the top of section of Adjusted Gross Income breakdowns. This means that it cannot be easily sorted, or joined to a geospatial dataset, without some intense modification. First a copy of the AGI Class needs to be created so it can be made into the Town Names.

In [11]:
towntax['Year'] = year
towntax['Town'] = towntax['AGIClss']

In [12]:
# Pulls list of unique values in AGIClss
# TownList = pd.unique(towntax['AGIClss'])

# Manually set a list of what are not valid town names, this might need to be updated if the brackets change in the future
delvalue =  ['Loss or None', '0.01 - 4999', '5000 - 9999','10000 - 14999', '15000 - 19999', '20000 - 24999',
             '25000 - 29999','30000 - 34999', '35000 - 39999', '40000 - 44999', '45000 - 49999', '50000 - 59999', 
             '60000 - 74999', '75000 - 99999', '100000 - 149999', '150000 +', '0.01 - 9999', '10000 - 19999', 
             '20000 - 29999','30000 - 39999', '40000 - 49999', '50000 - 74999', '75000 +','Grand Total']


In [13]:
# Creates a list of town names by excluding anything that is in delvalue list
townlist = towntax['Town']
townlist = [x for x in townlist if x not in delvalue]
townlist

['Addison',
 'Albany',
 'Alburgh',
 'Andover',
 'Arlington',
 'Athens',
 'Bakersfield',
 'Baltimore',
 'Barnard',
 'Barnet',
 'Barre City',
 'Barre Town',
 'Barton',
 'Belvidere',
 'Bennington',
 'Benson',
 'Berkshire',
 'Berlin',
 'Bethel',
 'Bloomfield',
 'Bolton',
 'Bradford',
 'Braintree',
 'Brandon',
 'Brattleboro',
 'Bridgewater',
 'Bridport',
 'Brighton',
 'Bristol',
 'Brookfield',
 'Brookline',
 'Brownington',
 'Brunswick',
 'Burke',
 'Burlington',
 'Cabot',
 'Calais',
 'Cambridge',
 'Canaan',
 'Castleton',
 'Cavendish',
 'Charleston',
 'Charlotte',
 'Chelsea',
 'Chester',
 'Chittenden',
 'Clarendon',
 'Colchester',
 'Concord',
 'Corinth',
 'Cornwall',
 'Coventry',
 'Craftsbury',
 'Danby',
 'Danville',
 'Derby',
 'Dorset',
 'Dover',
 'Dummerston',
 'Duxbury',
 'East Haven',
 'East Montpelier',
 'Eden',
 'Elmore',
 'Enosburg',
 'Essex Junction',
 'Essex Town',
 'Fair Haven',
 'Fairfax',
 'Fairfield',
 'Fairlee',
 'Fayston',
 'Ferrisburgh',
 'Fletcher',
 'Franklin',
 'Georgia',
 

In [14]:
# Iterate through the list and if the value matches the townlist, return that value, otherwise return the previous value

data = towntax['Town']
data = list(data)

newname = []
index1= 0
index2= -1 

for x in data:
    if data[index1] in townlist:
        newname.append(data[index1])
    else:
        newname.append(newname[index2])
    index1+=1
    index2+=1
    
    

In [15]:
# Add newly created town names to the town column
towntax['Town'] = newname

In [16]:
# Originaly the town name was on it's own row, this removes those and just leaves valid information. it also removed those footnotes
towntax = towntax[(towntax['Return'].notnull())]
towntax.head(25)

,AGIClss,Return,Exempt,MarJnt,Single,MarSep,HdHous,AGI,FedTI,VTI,NetVTax,Year,Town
1,Loss or None,17,23,7,8,2,0,-628514,0,0,2,2015,Addison
2,0.01 - 4999,226,150,9,206,1,10,614995,4338,3788,169.298,2015,Addison
3,5000 - 9999,220,195,7,196,0,17,1609364,108313,107526,3569.65,2015,Addison
4,10000 - 14999,177,204,14,148,4,11,2174646,377416,377425,12762,2015,Addison
5,15000 - 19999,179,232,9,144,7,19,3147689,1121572,1115947,35783.7,2015,Addison
6,20000 - 24999,157,222,15,116,4,22,3534548,1516449,1512012,49683.7,2015,Addison
7,25000 - 29999,156,206,12,123,6,15,4291183,2329438,2333222,78993.8,2015,Addison
8,30000 - 34999,165,264,28,110,7,20,5318610,2934351,2925775,98221.8,2015,Addison
9,35000 - 39999,114,182,24,77,3,10,4267105,2499161,2482740,82691.9,2015,Addison
10,40000 - 44999,72,111,14,40,7,11,3050192,2004670,2002328,65859.7,2015,Addison


## Column Order

In order for this data to be intuative, changing column order in necessary. First year, then town and then the AGI Class means that this data can be cut numerious ways

In [18]:
towntotal = towntax[(towntax['AGIClss'] == "Grand Total")]
towntax = towntax[(towntax['AGIClss'] != "Grand Total")]


In [17]:
# Reorders the columns
towntax = towntax[['Year', 'Town','AGIClss', 'Return', 'Exempt', 'MarJnt', 'Single', 'MarSep', 'HdHous',
       'AGI', 'FedTI', 'VTI', 'NetVTax']]
towntotal = towntotal[['Year', 'Town','Type', 'Return', 'Exempt', 'MarJnt', 'Single', 'MarSep', 'HdHous',
       'AGI', 'FedTI', 'VTI', 'NetVTax']]

In [19]:
towntax.tail(25)

,Year,Town,AGIClss,Return,Exempt,MarJnt,Single,MarSep,HdHous,AGI,FedTI,VTI,NetVTax
4004,2015,Woodstock,30000 - 34999,95,135,17,64,4,10,3068741,1632199,1616534,53635.6
4005,2015,Woodstock,35000 - 39999,96,156,31,55,1,9,3612277,1891024,1877645,62691.7
4006,2015,Woodstock,40000 - 44999,68,103,15,46,3,4,2882571,1782382,1758462,60249.5
4007,2015,Woodstock,45000 - 49999,69,98,18,48,2,1,3281552,2128041,2152724,74186.3
4008,2015,Woodstock,50000 - 59999,102,171,37,56,1,8,5603713,3560433,3608703,133277
4009,2015,Woodstock,60000 - 74999,121,241,59,46,2,14,8073350,5182144,5243464,198467
4010,2015,Woodstock,75000 - 99999,151,301,87,50,5,9,13087296,8999524,9093028,371118
4011,2015,Woodstock,100000 - 149999,181,442,143,27,3,8,21671751,15308079,1.59563e+07,694187
4012,2015,Woodstock,150000 +,199,495,158,30,5,6,59560673,44810293,50555276,2.8567e+06
4015,2015,Worcester,Loss or None,*,*,*,*,*,*,*,*,*,*


In [20]:
towntotal.tail(25)

,Year,Town,AGIClss,Return,Exempt,MarJnt,Single,MarSep,HdHous,AGI,FedTI,VTI,NetVTax
3655,2015,West Fairlee,Grand Total,265,506,121,114,5,25,14142081,9341224,9557975,437060
3665,2015,West Haven,Grand Total,113,207,52,52,2,7,5643380,3752306,3746931,171414
3683,2015,West Rutland,Grand Total,1280,2192,446,686,16,132,54466315,34664499,35104961,1.43378e+06
3701,2015,West Windsor,Grand Total,548,982,268,236,11,33,52799334,38300037,41444937,2.13732e+06
3711,2015,Westfield,Grand Total,267,465,117,131,4,15,12516214,7840678,8209772,361403
3729,2015,Westford,Grand Total,1016,1930,494,458,15,49,83741884,62302235,63756938,3.24156e+06
3747,2015,Westminster,Grand Total,1341,2391,564,647,28,102,73759235,51140502,52304461,2.4132e+06
3757,2015,Westmore,Grand Total,175,303,81,73,5,16,10722716,6751948,6561407,317221
3767,2015,Weston,Grand Total,313,536,138,149,8,18,25286557,18083846,19933518,971457
3785,2015,Weybridge,Grand Total,407,813,215,167,7,18,37719750,25977758,28220765,1.48923e+06


## Appending to Running Database

The point of all of this is to make adding multiple years of data together almost effortless. For the very first year the running data, stored as a csv, needs to be created. After that all you have to do is change the year at the top and rerun the kernel and it should automatically add the year selected to the running database  

In [21]:
if cumltowntax.empty:
    cumltowntax = towntax 
else:
    cumltowntax = cumltowntax.append(towntax, ignore_index=True)
    

if cumltowntotal.empty:
    cumltowntotal = towntotal
else:
    cumltowntotal = cumltowntotal.append(towntotal, ignore_index=True)

In [22]:
cumltowntax.tail(20)

,Year,Town,AGIClss,Return,Exempt,MarJnt,Single,MarSep,HdHous,AGI,FedTI,VTI,NetVTax
38864,2015,Woodstock,60000 - 74999,121,241,59,46,2,14,8073350,5182144,5243464,198467
38865,2015,Woodstock,75000 - 99999,151,301,87,50,5,9,13087296,8999524,9093028,371118
38866,2015,Woodstock,100000 - 149999,181,442,143,27,3,8,21671751,15308079,1.59563e+07,694187
38867,2015,Woodstock,150000 +,199,495,158,30,5,6,59560673,44810293,50555276,2.8567e+06
38868,2015,Worcester,Loss or None,*,*,*,*,*,*,*,*,*,*
38869,2015,Worcester,0.01 - 4999,34,30,5,26,1,2,85247,7784,7027,250
38870,2015,Worcester,5000 - 9999,40,39,4,35,0,1,302770,6953,6953,556
38871,2015,Worcester,10000 - 14999,34,50,6,21,1,6,426161,52720,52527,1816
38872,2015,Worcester,15000 - 19999,29,48,7,20,0,2,517525,142329,140378,4722.6
38873,2015,Worcester,20000 - 24999,29,47,9,17,1,2,655138,219883,219790,6960.24


In [23]:
cumltowntotal.tail(20)

,Year,Town,AGIClss,Return,Exempt,MarJnt,Single,MarSep,HdHous,AGI,FedTI,VTI,NetVTax
2784,2015,Westford,Grand Total,1016,1930,494,458,15,49,83741884,62302235,63756938,3.24156e+06
2785,2015,Westminster,Grand Total,1341,2391,564,647,28,102,73759235,51140502,52304461,2.4132e+06
2786,2015,Westmore,Grand Total,175,303,81,73,5,16,10722716,6751948,6561407,317221
2787,2015,Weston,Grand Total,313,536,138,149,8,18,25286557,18083846,19933518,971457
2788,2015,Weybridge,Grand Total,407,813,215,167,7,18,37719750,25977758,28220765,1.48923e+06
2789,2015,Wheelock,Grand Total,314,583,146,147,4,17,13967622,8647684,8775742,369384
2790,2015,Whiting,Grand Total,220,409,93,103,3,21,9185783,5680186,5708113,235084
2791,2015,Whitingham,Grand Total,635,1165,275,301,6,53,32206847,20957891,21524543,854763
2792,2015,Williamstown,Grand Total,1762,3137,708,849,34,171,81713808,53753962,54259961,2.29176e+06
2793,2015,Williston,Grand Total,4812,8882,2144,2302,77,289,4.30925e+08,3.23556e+08,3.35449e+08,1.77776e+07


In [24]:
# export to a csv, if the index is not set to false it will add an unnamed column with the original index which will need to be deleted individually
cumltowntax.to_csv("cumltowntaxincome.csv", index=False)
cumltowntotal.to_csv("cumltowntotal.csv", index=False)

## Adding to a Shapefile

Once all the years are added together it is time to join them with the Town Boundaries Shapefile in whichever GIS program you prefer. In the CSV there is a set of data that is marked in the Town column as Surpressed/Not Stated which cannot be joined and will be marked as ignored. It is important not to lose that data, while it is small it might be important given a specific problem. 